In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from keras_tuner import RandomSearch

In [ ]:
# Function for hyperparameterization

def build_model(hp):
    model = Sequential()
    
    # Define the number of layers
    for i in range(hp.Int('num_layers', 1, 5)):
        model.add(Dense(
            units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32),
            activation=hp.Choice(f'activation_{i}', values=['relu', 'tanh', 'sigmoid'])
        ))
    
    model.add(Dense(1, activation="linear"))

    lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    optimizer = Adam(learning_rate=lr)

    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

In [ ]:
# Reading input data to train the model
cols = ['time', 'longitude', 'latitude', 'xco2', 'u10', 'v10', 'd2m', 't2m', 'sp', 'cams', 'cams2', 'odiac', 'ndvi', 'landscan', 'gfed']
f = 'training_data_0p75.csv'
df = pd.read_csv(f, names=cols)
df['time'] = pd.to_datetime(df['time'])
df.dropna(inplace=True)

# Select input features and target variable
X = df[['u10', 'v10', 'cams', 'cams2', 'odiac', 'ndvi', 'landscan', 'gfed']]
y = df[['xco2']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

# Set up the tuner
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=50,
    executions_per_trial=3,
    directory='training',
    project_name='xco2_dnn'
)

# Perform the hyperparameter search
tuner.search(X_train_scaled, y_train, epochs=3000, validation_data=(X_test_scaled, y_test), batch_size=500)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the optimal hyperparameters and train it
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train_scaled, y_train, epochs=1000, batch_size=500, validation_data=(X_test_scaled, y_test))

# The best hyperparameters
print(f"""
The hyperparameter search is complete. The optimal hyperparameters are:
- Number of layers: {best_hps.get('num_layers')}
- Learning rate: {best_hps.get('learning_rate')}
""")
for i in range(best_hps.get('num_layers')):
    print(f" - Units in layer {i+1}: {best_hps.get(f'units_{i}')}")
    print(f" - Activation function in layer {i+1}: {best_hps.get(f'activation_{i}')}")

In [ ]:
# Training model using optimal parameters
#optimizer=Adam(lr=1e-3)
optimizer = Adam(learning_rate=1e-3)

# Define the neural network model
model = Sequential()
model.add(Dense(256, activation="relu", input_dim=X_train.shape[1]))
model.add(Dense(64, activation="relu"))
model.add(Dense(8, activation="relu"))
model.add(Dense(1))
model.compile(optimizer=optimizer, loss='mean_squared_error')

history = model.fit(X_train_scaled, y_train, epochs=3000, batch_size=500, validation_data=(X_test_scaled, y_test))

# Save the model
model.save('xco2_dnn_model.h5')